In [ ]:
import numpy as np
import pandas as pd
import DTA.polarDensity_helper as pc
from pathlib import Path
import matplotlib.pyplot as plt
from DTA.enrichment_plotters import polar_plot, get_file_list, get_helices, read_rep
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
plt.rcParams['axes.grid'] = False 

# Important User Settings:

In [ ]:
lipids = ["POPG", "POPE"]
leaflets = ['low', 'upp']
root = Path("/home/ems363/Projects/ELIC_PE_PG/aggregated/")
replicas = ["rep1", "rep3", "liam"]

enrichments = None
counts = None
helices_upr, helices_lwr = get_helices(root.joinpath(replicas[2]))
for rep in replicas:
    reppath = root.joinpath(rep)
    file_list = get_file_list(reppath, lipids)
    assert len(file_list) > 0, "ERROR: No files found!"
    rad, dr, dth, theta, radius, frames, Ntheta = pc.Coord_Get(file_list[0])
    cts, rich = read_rep(file_list, lipids, leaflets, enrich=True)
    if counts is None:
        counts = cts
        enrichments = rich
    else:
        counts = counts+cts
        enrichments = enrichments+rich
    

counts = counts/len(replicas)
enrichments = enrichments/len(replicas)
thetas = np.unique(theta.flatten())

# Do the plotting

In [ ]:
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('seismic', 256)
newcolors = np.concatenate([depleted(np.linspace(0.3, 0.5, 128)), enriched(np.linspace(0.5,0.8,128))])
my_cmap = ListedColormap(newcolors)

In [ ]:
#enrichments.at['POPE', 'low'][0:1,:]=0
fig, axes = polar_plot(enrichments, 
                       theta, 
                       radius, 
                       lipids, 
                       helices_lwr, 
                       helices_upr, 
                       colorbychain=False, 
                       vmin=0.75, 
                       vmax=2, 
                       vmid=1,
                       figheight=12,
                       figwidth=12,
                       cmap=my_cmap)

axes[0] = plot_sites(axes[0], thetas, 4, 3, 24, 6)
axes[1] = plot_sites(axes[1], thetas, 8, 4, 22, 10)

plt.savefig(root.joinpath("ELIC_enrichments.pdf"))
plt.show()